In [ ]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt

In [ ]:
#iterate over files
content = []
files = glob.glob('../zillow_files/apartments_scraped_*')
for file in files:
    initial_load = pd.read_csv(file, index_col=0)
    content.append(initial_load)
print(content)

df = pd.concat(content)

# convert date column from object to datetime
df['Date_Webscraped'] = df['Date_Webscraped'].astype('datetime64')

In [ ]:
#check for null values
df.isna().sum()

In [ ]:
 # preliminary stats-- what's the avg price per bedroom, 
 # and what's the distribution of apartments with different # of bedrooms
 # clearly a lot more apartments with 2 bedrooms are listed than other apartment sizes
 # does the date the data was first scraped affect any of these stats

beds_df = df.groupby(['Beds','Date_Webscraped']).agg({'Price':np.mean,"Beds":np.size})
print(beds_df)